In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.transforms import ToTensor

# Image Pre-Processessing

In [3]:
def resize_images_in_dir(raw_data_path:Path, width:int, height:int):
	for img_path in raw_data_path.iterdir():
		img = Image.open(img_path)
		img = img.resize((width, height), Image.ANTIALIAS)
		img.save(img_path)

# Dataset Processing

In [4]:
class FaceClassifierDataset(Dataset):
    def __init__(self, annotations_file, img_dir):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = self.img_dir / str(self.img_labels.iloc[idx, 0])
        image = read_image(img_path.as_posix())
        image = self.transform(image)
        label = self.img_labels.iloc[idx, 1]
        label = self.target_transform(label)
        return image, label

    def transform(self, image):
        return ToTensor(image)

    def target_transform(self, label):
        if label.upper() == "TAYLOR":
            return 0
        elif label.upper() == "ROCK":
            return 1
        elif label.upper() == "QUENTIN":
            return 2

# CNN Network

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
root_data_path = Path("/home/hpc/kurlanl1/CSC-380/CSC380-Artificial-Intelligence/faces_dataset")
train_data_path = root_data_path / "train"
test_data_path = root_data_path / "train"

train_dataset = FaceClassifierDataset(train_data_path / "y" / "y.csv", 
					test_data_path / "X")
test_dataset = FaceClassifierDataset(test_data_path / "y" / "y.csv", 
					test_data_path / "X")

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [7]:
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [9]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: image::read_file() Expected a value of type 'str' for argument '_0' but instead found type 'PosixPath'.
Position: 0
Value: PosixPath('/home/hpc/kurlanl1/CSC-380/CSC380-Artificial-Intelligence/faces_dataset/train/X/t1.jpg')
Declaration: image::read_file(str _0) -> (Tensor _0)
Cast error details: Unable to cast Python instance to C++ type (compile in debug mode for details)